### Calwindow clustering

This notebook contains different dimensionality reductions techniques tried such as Singular Value Decomposition, Discrete fourier transforms and Discrete Wavelet transforms on only calibration window of the time series data. Clustering methods such as Kmeans, DBSCAN and Birch.

In [ ]:
import pandas as pd

In [ ]:
ecd_ts = pd.read_csv('/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/TimeSeries/ecd_TS.csv')
un_ts = pd.read_csv('/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/TimeSeries/US_TS.csv')

In [ ]:
len(ecd_ts)

In [ ]:
ecd_ts['Label']= "ecdcontact"

In [ ]:
ecd_ts

In [ ]:
len(un_ts)

In [ ]:
## ecdcontacts are not in unsuccessful readings
un_ts[un_ts['TestId'] == 8071094]

In [ ]:
un_ts['Label'] = 'Unsuccessful'

In [ ]:
data = pd.concat([ecd_ts,un_ts])

In [ ]:
data

In [ ]:
data[data['TestId'] == 9610462]

In [ ]:
data[data['TestId'] == 9610647]

In [ ]:
## these two are outliers as seen from predictors file need to remove them 

In [ ]:
data = data.drop([data.index[140] , data.index[141]])

In [ ]:
# Crosschecking
# data[data['TestId'] == 9610462]
data[data['TestId'] == 9610647]

In [ ]:
data.isna().sum()

In [ ]:
test = data

In [ ]:
test

In [ ]:
caldata = test.iloc[:,:1005]

In [ ]:
caldata['Label'] = test['Label'] 

In [ ]:
caldata

In [ ]:
caldata[caldata.isna().any(axis=1)]

In [ ]:
## dropecdg na values to see 
caldatatest = caldata.dropna()
caldatatest = caldatatest.reset_index(drop=True)

In [ ]:
caldatatest

In [ ]:
caldatatest.to_csv("/Users/neethug/Desktop/Neethu/Course/DATA599/Project/finaldata/caltest.csv",index = False)

In [ ]:
A = caldatatest.iloc[:,1:-1]

### Singular Value Decomposition for dimensionality reduction

The Singular-Value Decomposition, or SVD for short, is a matrix decomposition method for reducing a matrix to its constituent parts in order to make certain subsequent matrix calculations simpler.

SVD takes in a matrix and return U S and VT

In [ ]:
from sklearn.decomposition import TruncatedSVD

# svd
svd = TruncatedSVD(n_components=2)
svd.fit(A)
result = svd.transform(A)
print(result)

In [ ]:
df = pd. DataFrame(result, columns=['SVD1', 'SVD2'])

In [ ]:
df['TestID'] = caldatatest['TestId']

In [ ]:
df['Label'] = caldatatest['Label']

In [ ]:
df

In [ ]:
!pip install altair

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(df).mark_circle(size=60).encode(
    x='SVD1',
    y='SVD2',
    color = 'Label',
    tooltip=['TestID']
)

## Discrete Fourier transform 

To derive frequency domain prespective of the time series

Didnt work for entire dataset as it containc nan values, and the dft returns nan values for all columns.

In [ ]:
from scipy.fft import fft, ifft

In [ ]:
y = fft(A,n=2)

DFT provides frequency domain information on the entire waveform if we give n=2 wont we miss out on the information stored in further columns?

if we do consider all the columns then we might again want to apply reduction technique?

In [ ]:
len(y)

In [ ]:
df2 = pd. DataFrame(abs(y),columns = ['DFT1','DFT2'])

In [ ]:
df2

In [ ]:
df2['TestID'] = caldatatest['TestId']

In [ ]:
df2['Label'] = caldatatest['Label']

In [ ]:
df2.columns

In [ ]:
alt.Chart(df2).mark_circle(size=60).encode(
    x='DFT1',
    y='DFT2',
    color = 'Label',
    tooltip=['TestID']
)

PLotting wavefroms before and after trasformation to see what it represents

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
caldatatest.iloc[:1,1:-1]

In [ ]:
len(x1)

In [ ]:
len(y1)

In [ ]:
x1 = caldatatest.columns.to_list()
x1.remove('TestId')
x1.remove('Label')

In [ ]:
y1 = caldatatest.iloc[0][1:-1].values
plt.plot(x1,y1)

In [ ]:
x2 = df2.columns.to_list()
x2.remove('TestID')
x2.remove('Label')
y2 = df2.iloc[0][0:2].values

In [ ]:
plt.plot(x2,y2)

## Fourier transform works when no variation in time happens, when we are in the frequency-domain we lose this dependency

In some way we can think of the Fourier transform as a trade-off between time information and frequency information. By taking a FT of a time signal, all time information is lost in return for frequency information

## Discrete Wavelet Transform

This considers both frequency and time aspects of the waveform ,

A discrete wavelet transform (DWT) is a transform that decomposes a given signal into a number of sets, where each set is a time series of coefficients describing the time evolution of the signal in the corresponding frequency band.

In [ ]:
!pip install PyWavelets

In [ ]:
import pywt

In [ ]:
res, res1 = pywt.dwt(A,wavelet='Haar')

In [ ]:
df = pd.DataFrame(res)

In [ ]:
df

In [ ]:
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
pca = PCA().fit(df)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlim(0,7,1)
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')

In [ ]:
pca = PCA(n_components=2)
pc = pca.fit_transform(df)

In [ ]:
dt = pd.DataFrame(pc, columns = ['PC1','PC2'])

In [ ]:
dt

In [ ]:
dt['Label'] = caldatatest['Label']

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

# Instantiate the clustering model and visualizer
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12))

In [ ]:
visualizer.fit(dt)      
visualizer.show()

In [ ]:
plt.scatter(dt['PC1'],dt['PC2'])

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

In [ ]:
alt.Chart(dt).mark_circle(size=60).encode(
    x='PC1',
    y='PC2',
    color = 'Label'
)

In [ ]:
X = dt.iloc[:,:-1]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmodel = KMeans(n_clusters=3)

In [ ]:
kmodel.fit(X)

In [ ]:
yhat = kmodel.predict(X)

In [ ]:
dt['cluster'] = yhat

In [ ]:
dt[dt['cluster'] == 0].groupby('Label').count()

In [ ]:
dt

In [ ]:
X['cluster'] = yhat

In [ ]:
X

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


X['cluster'] = le.fit_transform(X['cluster'])

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,12))

In [ ]:
visualizer.fit(X)      
visualizer.show()

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
Dmod = DBSCAN(eps=0.30, min_samples=10)

In [ ]:
yhat = Dmod.fit_predict(X)

In [ ]:
unique(yhat)

In [ ]:
X['newcluster']= yhat

In [ ]:
X['cluster'] == X['newcluster']

In [ ]:
df3 = df2

In [ ]:
df3['DFT1'] = res[0]

In [ ]:
df3['DFT2'] = res[1]

In [ ]:
df3

In [ ]:
alt.Chart(df3).mark_circle(size=60).encode(
    x='DFT1',
    y='DFT2',
    color = 'Label',
    tooltip=['TestID']
)

In [ ]:
## fitting few machine learning algorithms to see the clusters forming.

In [ ]:
import sklearn
from sklearn.cluster import Birch

In [ ]:
model = Birch(threshold=0.01, n_clusters=2)

In [ ]:
model.fit(X)

In [ ]:
from modwt import modwt

In [ ]:
result = modwt(A['0.2'], 'haar', 6)

In [ ]:
result

In [ ]:
caldatatest[caldatatest['TestId'] == 10206668]

In [ ]:
y = caldatatest.iloc[206][1:-1].values

In [ ]:
plt.plot(x1,y)

In [ ]:
caldatatest[caldatatest['TestId'] == 10343506]

In [ ]:
y2 = caldatatest.iloc[6691][1:-1].values

In [ ]:
plt.plot(x1,y2)